In [5]:
## We will first import the necessary libraries.

import pandas as pd
import numpy as np
import time
from selenium import webdriver
import requests

In [6]:
## We define our driver. We are working with selenium which needs chromedriver. 
## You can download it from https://chromedriver.chromium.org/downloads
## If you are using some other browser, you can download it's driver from a simple search on google.com
## We define our driver and set the executable_path to the path of out chromedriver.

driver = webdriver.Chrome(executable_path='/Users/Vatanpatel/Documents/greendeck/chromedriver')

In [4]:
## Read the csv that contains product reference numbers for oneill.com.

df = pd.read_csv('Greendeck Business Analyst Assignment Task 4 - Sheet1.csv')
df.head()

,Brand,Reference,Google Search Code,Category,Name,Product Page URL,Price(Euros)
0,O'NEILL,0A4972-9950 BLACK AOP,0A4972-9950,Sandals,FB ARCH PRINT SANDALS,NaN,NaN
1,O'NEILL,9P1070-5056 Ink Blue,9P1070-5056,Jackets,LB CHARGER PARKA,NaN,NaN
2,O'NEILL,9P1070-6077 Winter Mos,9P1070-6077,Jackets,LB CHARGER PARKA,NaN,NaN
3,O'NEILL,8P1072-5056 Ink Blue,8P1072-5056,Jackets Technical,LB TRANSIT JACKET,NaN,NaN
4,O'NEILL,8P1072-5112 Surf Blue,8P1072-5112,Jackets Technical,LB TRANSIT JACKET,NaN,NaN


In [8]:
## We use two methods to scrape this data and see which one is performing best. 
## We use serpstack api first. We have 65 free requests remaining so we will only make 65 requests.

access_key = '*********************'    ## you have to enter your access key after login into serpstack.
price = []    ## Define an empty price list
prod = []    ## Define an empty prod list
for i in np.arange(65):    ## We choose first 65 products in the dataframe.
    try:
        params = {
        'access_key': access_key,
        'query': '{}'.format(df['Google Search Code'][i])    ## We make the query for each Google Search Code in df
        }
        api_result = requests.get('http://api.serpstack.com/search', params)    ## We get all the parameters provided by our request
        api = api_result.json()['organic_results'][0]['url']    ## We select the url for the first google result.
        driver.get(api)    ## We open the url in our browser using driver.
        ## Now here we select the css selector path for price on each product page and add the price in our price list.
        price.append(driver.find_element_by_css_selector('div.d-md-block:nth-child(1) > div:nth-child(2) > div:nth-child(1) > span:nth-child(1) > span:nth-child(4) > span:nth-child(1)').text)
        prod.append(driver.current_url)    ## We also append the url into our prod list.
    except:
        price.append('NaN')    ## if we do not find any product, we do not want the scraping to stop. So we return NaN for them.
        prod.append('NaN')

In [ ]:
## we create a dataframe for our results.


serpstack.reset_index(inplace = True)
serpstack = pd.DataFrame(prod, price)
serpstack.columns = ['Price', 'URL']


In [7]:
## In our second scraping method, we will use selenium for that. We again define two empty lists.

price2 = []
prod2 = []
for i in np.arange(129):    ## We will search for all the rows given to us because selenium has no limit.
    try:    
        ## We take the google search URL and insert our google search code as a query.
        driver.get('https://www.google.com/search?source=hp&ei=wRt3X8TIKYCc4-EPhd2ToAM&q={}'.format(df['Google Search Code'][i]))
        ## We find the first search result and click on it. We wait for 2 seconds for the page to load.
        driver.find_element_by_css_selector('div.g:nth-child(1) > div:nth-child(2) > div:nth-child(1) > a:nth-child(1) > h3:nth-child(2) > span:nth-child(1)').click()
        time.sleep(2)
        url = driver.current_url    ## We save current browser url in url.
        driver.get(url)    ## We open the url in our browser
        prod2.append(driver.current_url)    ## We add this url into our prod2 list
        ## here if the price is available, we add it to our price2 list else add NaN to our list. We do the same for prod2.
        try:
            price2.append(driver.find_element_by_css_selector('body > div.page > div.container.product-detail.product-wrapper > div.row.product-detail__row-main > div.product-detail__basic-info.col-12.col-sm-6.col-md-4.col-xl-3.mb-sm-3.order-2 > div:nth-child(2) > div:nth-child(1) > div > div > span > span.sales > span').text)
        except:
            price2.append('NaN')
    except:
        prod2.append('NaN')
        price2.append('NaN')

In [37]:
## We again create a dataframe for our second results.

serpstack2 = pd.DataFrame(prod2, price2)
serpstack2.reset_index(inplace = True)
serpstack2.columns = ['Price', 'URL']
serpstack2

,Price,URL
0,"€16,09",https://www.oneill.com/eu/en/arch-print-sandal...
1,"€76,99",https://www.oneill.com/eu/en/charger-parka-jac...
2,"€76,99",https://www.oneill.com/eu/en/charger-parka-jac...
3,NaN,NaN
4,NaN,NaN
...,...,...
124,"€25,99",https://www.oneill.com/eu/en/summer-tanktop/0A...
125,"€ 25,99",https://www.oneill.com/de/en/summer-tanktop/0A...
126,NaN,https://www.oneill.com/be/nl/mountain-gaze-lon...
127,"€ 12,59",https://www.oneill.com/fr/en/palm-t-shirt/0A73...


In [36]:
len(serpstack2[serpstack2.Price == 'NaN']) ## We check for values not available, We have 22 products missing in our second method.

22

In [59]:
## We have most of the values predicted by our second method. We choose the missing values from our second result.

index = serpstack2[serpstack2.Price == 'NaN'].index

In [68]:
## Let's check our missing values in our first result.  
## In our first result, we got results for only the first 65 products. We crop our index for values below 65.

index2 = index[:13]

In [73]:
## We look for the missing values in the first result. We find that we get 4 results in first method 
## which were missing in second result.

serpstack.iloc[index2][['Price', 'URL']]

,Price,URL
3,NaN,NaN
4,NaN,NaN
22,NaN,NaN
27,NaN,NaN
31,NaN,NaN
49,"€34,99",https://www.oneill.com/eu/en/hooded-full-zip-s...
50,NaN,NaN
52,"€27,99",https://www.oneill.com/eu/en/crew-ski-fleece/9...
56,NaN,NaN
57,NaN,NaN


In [74]:
## Let's impute these missing values into our second result.

serpstack2.iloc[index2] = serpstack.iloc[index2][['Price', 'URL']]

In [75]:
## We see that the values have been imputed.

serpstack2.iloc[index2]

,Price,URL
3,NaN,NaN
4,NaN,NaN
22,NaN,NaN
27,NaN,NaN
31,NaN,NaN
49,"€34,99",https://www.oneill.com/eu/en/hooded-full-zip-s...
50,NaN,NaN
52,"€27,99",https://www.oneill.com/eu/en/crew-ski-fleece/9...
56,NaN,NaN
57,NaN,NaN


In [92]:
## Now finally we impute these values into our orignal dataframe. 

df[['Product Page URL', 'Price(Euros)']] = serpstack2[['URL', 'Price']]

In [108]:
## Here we define a function to edit the URL by changing the region and language to France and English respectively

def region(url, region, lang):
    a = url.split('/')
    a[3:5] = ['fr', 'en']
    a = '/'.join(a)
    return(a)

In [109]:
## We check if the function is working properly.

region('https://www.oneill.com/eu/fr/arch-print-sandals/0A4972-9950-32.html', 'fr', 'en')

'https://www.oneill.com/fr/en/arch-print-sandals/0A4972-9950-32.html'

In [146]:
## We ignore the NAN or NaN values and edit the rest of the urls using the .apply() method.

df['URL'] = df['Product Page URL'][df['Product Page URL'].isna() == False][df['Product Page URL'] != 'NaN'].apply(lambda x: region(x, 'fr', 'en'))
df['Product Page URL'] = df['URL']
df.drop('URL', axis = 1, inplace = True)

In [153]:
## Now we finally save our final file. 

df.to_csv('Oneill_Final.csv', index = False)

In [154]:
pd.read_csv('Oneill_Final.csv')

,Brand,Reference,Google Search Code,Category,Name,Product Page URL,Price(Euros)
0,O'NEILL,0A4972-9950 BLACK AOP,0A4972-9950,Sandals,FB ARCH PRINT SANDALS,https://www.oneill.com/fr/en/arch-print-sandal...,"€16,09"
1,O'NEILL,9P1070-5056 Ink Blue,9P1070-5056,Jackets,LB CHARGER PARKA,https://www.oneill.com/fr/en/charger-parka-jac...,"€76,99"
2,O'NEILL,9P1070-6077 Winter Mos,9P1070-6077,Jackets,LB CHARGER PARKA,https://www.oneill.com/fr/en/charger-parka-jac...,"€76,99"
3,O'NEILL,8P1072-5056 Ink Blue,8P1072-5056,Jackets Technical,LB TRANSIT JACKET,NaN,NaN
4,O'NEILL,8P1072-5112 Surf Blue,8P1072-5112,Jackets Technical,LB TRANSIT JACKET,NaN,NaN
...,...,...,...,...,...,...,...
124,O'NEILL,0A7383-1940 WHITE AOP,0A7383-1940,Tees,LG SUMMER TANKTOP,https://www.oneill.com/fr/en/summer-tanktop/0A...,"€25,99"
125,O'NEILL,0A7383-4960 PINK AOP W,0A7383-4960,Tees,LG SUMMER TANKTOP,https://www.oneill.com/fr/en/summer-tanktop/0A...,"€ 25,99"
126,O'NEILL,8P7170-3350 Neon Tange,8P7170-3350,Tees,LG MOUNTAIN GAZE L/SLV T-SHIRT,https://www.oneill.com/fr/en/mountain-gaze-lon...,NaN
127,O'NEILL,0A7392-6082 Lily Pad,0A7392-6082,Tees,LG PALM T-SHIRT,https://www.oneill.com/fr/en/palm-t-shirt/0A73...,"€ 12,59"


In [157]:
## We quit the driver.

driver.quit()

In [ ]:
## I hope this was easy for you to follow through. If you have any comments or need any clarifications,
## write an email @ i12vatanp@iimidr.ac.in